In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier, StackingClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import pdfplumber
import docx
import joblib
import re
import pickle

In [2]:
resume_data = pd.read_csv('Enhanced_Resume_Dataset.csv')
# pd.set_option("display.max_rows", None)
resume_data

,Category,Resume,Experience (Years),Skills Count,Education Level,Certifications Count,Projects Count,Resume Score
0,Data Science,Skills * Programming Languages: Python (pandas...,0,13,Bachelor's,0,1,40
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,0,4,Bachelor's,0,0,12
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",0,9,Bachelor's,0,0,27
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,0,6,Bachelor's,0,11,29
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",0,3,Master's,0,0,9
...,...,...,...,...,...,...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...,0,2,Bachelor's,0,0,6
958,Testing,â Willingness to accept the challenges. â ...,0,2,Bachelor's,0,0,6
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",0,1,Bachelor's,3,0,9
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,0,1,Bachelor's,0,4,7


In [4]:
resume_data['Category'].unique()

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [8]:
encoder = LabelEncoder()
resume_data["Category"] = encoder.fit_transform(resume_data["Category"])

In [11]:
with open("category_encoder1212.pkl", "wb") as f:
    pickle.dump(encoder, f)

In [10]:
resume_data['Category'].unique()

array([ 6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8,
       17, 19,  7, 13, 10,  9,  3, 23])

In [14]:
resume_data[resume_data["Resume Score"] == 40].shape[0]

4

FOR JOB ROLE PREDICTION

In [3]:
tfidf = TfidfVectorizer(max_features=5000)
x = tfidf.fit_transform(resume_data['Resume']).toarray()
y = resume_data.iloc[:, 0] 

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
rf = RandomForestClassifier(n_estimators=300, max_depth=20)
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=300)

In [6]:
TrainingAccuracy = rf.score(x_train,y_train)*100
TestingAccuraccy = rf.score(x_test,y_test)*100
print(f"Training accuracy : {TrainingAccuracy:.2f}")
print(f"Testing accuracy : {TestingAccuraccy:.2f}")

Training accuracy : 100.00
Testing accuracy : 100.00


FOR JOB SCORE/ QUALITY PREDICTION

In [7]:
resume_data['Education Level'].unique()

array(["Bachelor's", "Master's", 'Other'], dtype=object)

In [8]:
le = LabelEncoder()
resume_data['Education Level'] = le.fit_transform(resume_data['Education Level'])

In [9]:
resume_data['Education Level'].unique()

array([0, 1, 2])

In [10]:
p = resume_data.iloc[:, [2, 3, 4, 5, 6]]
z = resume_data.iloc[:,-1]

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(p, z, test_size=0.2)

In [26]:
rfa = XGBRegressor()
rfa.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
TrainingAccuracy = rfa.score(X_train,Y_train)*100
TestingAccuraccy = rfa.score(X_test,Y_test)*100
print(f"Training accuracy : {TrainingAccuracy:.2f}")
print(f"Testing accuracy : {TestingAccuraccy:.2f}")

Training accuracy : 100.00
Testing accuracy : 100.00


FUNCTION COMBINING BOTH JOB ROLE AND RESUME SCORE PREDICTION

In [19]:
def extract_text_from_file(file_path):
    text = ""
    file_path_lower = file_path.lower()
    if file_path_lower.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    elif file_path_lower.endswith(".docx"):
        doc = docx.Document(file_path)
        text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
    else:
        return None

    return text.strip() if text.strip() else None

def extract_experience_safe(resume_text):
    experience_patterns = [
        r'(\d+)\s*[\+]?[\s-]*(?:years?|yrs?)\s*(?:of)?\s*experience',
        r'experience\s*(?:of)?\s*(\d+)\s*[\+]?[\s-]*(?:years?|yrs?)'
    ]
    for pattern in experience_patterns:
        match = re.search(pattern, resume_text, re.IGNORECASE)
        if match:
            try:
                return int(match.group(1))
            except ValueError:
                return 0
    return 0

common_skills = [
    "Python", "Java", "SQL", "Machine Learning", "Deep Learning", "AI",
    "C++", "JavaScript", "HTML", "CSS", "TensorFlow", "Keras", "Pandas",
    "Numpy", "Tableau", "Excel", "Power BI", "Flask", "Django",
    "R", "C#", "PHP", "Ruby", "Docker", "Kubernetes", "AWS", "Azure", "GCP",
    "Linux", "Unix", "Bash", "Scala", "Swift", "Objective-C", "Git", "Spark",
    "Hadoop", "NoSQL", "MongoDB", "PostgreSQL", "Oracle", "Jupyter", "REST APIs",
    "GraphQL"
]

def count_skills(resume_text):
    return sum(1 for skill in common_skills if skill.lower() in resume_text.lower())

def extract_education(resume_text):
    text_lower = resume_text.lower()
    if "phd" in text_lower:
        return "PhD"
    elif "master" in text_lower or "m" in text_lower:
        return "Master's"
    elif "bachelor" in text_lower or "b" in text_lower:
        return "Bachelor's"
    else:
        return "Other"

def count_certifications(resume_text):
    return len(re.findall(r"certified|certification|certificate", resume_text, re.IGNORECASE))

def count_projects(resume_text):
    return len(re.findall(r"project", resume_text, re.IGNORECASE))

education_map = {
    "Other": 0,
    "Bachelor's": 1,
    "Master's": 2,
    "PhD": 3
}

def predict_resume():
    file_path = input("Enter the full path of the resume file (PDF/DOCX): ").strip()
    text = extract_text_from_file(file_path)

    if text is None:
        print("❌ Unsupported file format or empty file. Please use PDF or DOCX.")
        return

    text_vectorized = tfidf.transform([text]).toarray()
    predicted_role = rf.predict(text_vectorized)[0]  # Classifier model

    experience = extract_experience_safe(text)
    skills_count = count_skills(text)
    education_level = extract_education(text)
    certifications_count = count_certifications(text)
    projects_count = count_projects(text)

    education_encoded = education_map.get(education_level, 0)

    input_features = np.array([[experience, skills_count, education_encoded, certifications_count, projects_count]])

    predicted_score = rfa.predict(input_features)[0]

    def categorize_score(score):
        if score >= 40:
            return "Good"
        elif 20 <= score < 40:
            return "Moderate"
        else:
            return "Bad"

    score_category = categorize_score(predicted_score)

    print("\n🔍 Predicted Job Role:", predicted_role)
    print(f"🔍 Predicted Resume Score: {predicted_score:.2f} → {score_category}")

In [20]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Mohd Amaan Khan Resume(new).pdf

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 27.41 → Moderate


In [21]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Firdous_new_cv_resume.pdf

🔍 Predicted Job Role: Python Developer
🔍 Predicted Resume Score: 33.30 → Moderate


In [22]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): my_resume.pdf

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 36.59 → Moderate


In [23]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX):  M.Shahnawaz Resume.pdf

🔍 Predicted Job Role: Automation Testing
🔍 Predicted Resume Score: 25.66 → Moderate


In [24]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): CV_Salman1.pdf

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 36.49 → Moderate


In [25]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Hammad Yusuf cv mam recent-converted-1 (1)_compressed.pdf

🔍 Predicted Job Role: Advocate
🔍 Predicted Resume Score: 5.96 → Bad


In [22]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Mohd Amaan Khan Resume(new).pdf

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 26.65 → Moderate


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [23]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Industry manager resume (1).pdf

🔍 Predicted Job Role: Health and fitness
🔍 Predicted Resume Score: 9.00 → Bad


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [24]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Mohd Amaan Khan Resume.docx

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 19.00 → Bad


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [25]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Firdous_new_cv_resume.docx

🔍 Predicted Job Role: Python Developer
🔍 Predicted Resume Score: 34.12 → Moderate


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [26]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): my_resume.docx

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 32.92 → Moderate


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [27]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): M.Shahnawaz Resume.docx

🔍 Predicted Job Role: Automation Testing
🔍 Predicted Resume Score: 13.30 → Bad


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [28]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): CV_Salman1.docx

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 39.94 → Moderate


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [29]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Hammad Yusuf cv mam recent-converted-1 (1)_compressed.docx

🔍 Predicted Job Role: Advocate
🔍 Predicted Resume Score: 6.00 → Bad


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [30]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Mohd Amaan Khan Resume(new).docx

🔍 Predicted Job Role: Data Science
🔍 Predicted Resume Score: 26.65 → Moderate


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [31]:
predict_resume()

Enter the full path of the resume file (PDF/DOCX): Industry manager resume.docx

🔍 Predicted Job Role: Health and fitness
🔍 Predicted Resume Score: 9.00 → Bad


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [28]:
with open("resume_score_model1.pkl", "wb") as f:
    pickle.dump(rfa,f)